<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_TL2_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Transfer Learning Fine-tuning

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

# import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# import random

# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory

## Setup

In [2]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-darkgrid')

### Constantes

In [3]:
SEMENTE = 2008193

DIR_TREINO    = '10_food_classes_all_data/train'
DIR_TREINO_10 = '10_food_classes_10_percent/train'
DIR_TREINO_1  = '10_food_classes_1_percent/train'
DIR_TESTE     = '10_food_classes_all_data/test'

TAMANHO_LOTE = 32
FORMATO_IMAGEM = (224, 224)
MODO_CLASSE = 'categorical'

FORMATO_ENTRADA = FORMATO_IMAGEM + (3,)
ATIVACAO = 'softmax'

PERDA = 'categorical_crossentropy'
METRICAS = ['accuracy']

# OTIMIZADOR = 'Adam'
# APRENDIZADO = 0.001

### Funções

In [4]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

--2023-01-22 12:41:48--  https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4041 (3.9K) [text/plain]
Saving to: ‘funcoes.py’

funcoes.py          100%[===================>]   3.95K  --.-KB/s    in 0s      

2023-01-22 12:41:48 (46.6 MB/s) - ‘funcoes.py’ saved [4041/4041]



In [5]:
from funcoes import criar_callback_tensorboard, grafico_perda_precisao_por_iteracao, unzip_data, walk_through_dir

## Dados

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip

unzip_data('10_food_classes_all_data.zip')
unzip_data('10_food_classes_10_percent.zip')
unzip_data('10_food_classes_1_percent.zip')

In [7]:
# walk_through_dir('10_food_classes_all_data')

### Importando os dados e tranformando-os em lotes

In [ ]:
dados_treino = image_dataset_from_directory(directory=DIR_TREINO,
                                            batch_size=TAMANHO_LOTE,
                                            image_size=FORMATO_IMAGEM,
                                            label_mode=MODO_CLASSE,
                                            seed=SEMENTE)

dados_teste = image_dataset_from_directory(directory=DIR_TESTE,
                                           batch_size=TAMANHO_LOTE,
                                           image_size=FORMATO_IMAGEM,
                                           label_mode=MODO_CLASSE,
                                           seed=SEMENTE)

dados_treino_10 = image_dataset_from_directory(directory=DIR_TREINO_10,
                                               batch_size=TAMANHO_LOTE,
                                               image_size=FORMATO_IMAGEM,
                                               label_mode=MODO_CLASSE,
                                               seed=SEMENTE)

dados_treino_1 = image_dataset_from_directory(directory=DIR_TREINO_1,
                                              batch_size=TAMANHO_LOTE,
                                              image_size=FORMATO_IMAGEM,
                                              label_mode=MODO_CLASSE,
                                              seed=SEMENTE)

In [11]:
# image_dataset_from_directory retorna um BatchDataset

# O primeiro tensor contém as imagens. DADOS NÃO ESCALONADOS.
# O tamanho do lote não está especificado, mesmo que tenha sido informado.
# Os canais de cores estão definidos pelo parâmetro color_mode='rgb' (default).

# O segundo tensor contém as classes em formato one-hot.

dados_treino

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [12]:
rotulos = dados_treino.class_names
# rotulos

### Criando modelo

Modelos pré-treinados: https://www.tensorflow.org/api_docs/python/tf/keras/applications